In [21]:
import sys; sys.path.insert(0, "/home/adriano/sthunder")
from sqlalchemy import create_engine, select, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import os
import numpy as np
import pandas as pd
from sthunder.database import db_schema as dbs

In [2]:
class Database:
    def __init__(self, **kwargs):
        self._base = declarative_base()
        self._user = kwargs.get('username', os.environ['USER_POSTGRES'])
        self._pass = kwargs.get('password', os.environ['PASS_POSTGRES'])
        self._port = kwargs.get('port', '5432')
        self._host = kwargs.get('host', '127.0.0.1')
        self._name = kwargs.get('database', 'sthunder')
        self._uri = f'postgresql://{self._user}:{self._pass}@{self._host}/' \
                    f'{self._name}'
        self.session = sessionmaker(bind=create_engine(self._uri, echo=False))()

In [12]:
db = Database()

In [ ]:
q = db.session.query(
    dbs.FlashSpatioTemporal.id, 
    dbs.State.name,
    dbs.FlashSpatioTemporal.total,
    dbs.FlashCoordinate.geom.st_astext(), 
    dbs.FlashDatetime.datetime
).join(
    dbs.FlashCoordinate, dbs.FlashSpatioTemporal.coords == dbs.FlashCoordinate.id, 
).join(
    dbs.State, func.st_contains(dbs.State.geom, dbs.FlashCoordinate.geom)
).join(
    dbs.FlashDatetime, dbs.FlashSpatioTemporal.time == dbs.FlashDatetime.id
).all()

2021-09-05 23:18:05,949 INFO sqlalchemy.engine.base.Engine SELECT flash_spatio_temporal.id AS flash_spatio_temporal_id, state.name AS state_name, flash_spatio_temporal.total AS flash_spatio_temporal_total, ST_AsText(flash_coordinate.geom) AS "ST_AsText_1", flash_datetime.datetime AS flash_datetime_datetime 
FROM flash_spatio_temporal JOIN flash_coordinate ON flash_spatio_temporal.coords = flash_coordinate.id JOIN state ON ST_Contains(state.geom, flash_coordinate.geom) JOIN flash_datetime ON flash_spatio_temporal.time = flash_datetime.id
2021-09-05 23:18:05,950 INFO sqlalchemy.engine.base.Engine {}


In [24]:
q[0]

(32965449, 'Amazonas', 'POINT(-61 -5.5)', datetime.datetime(2020, 1, 1, 4, 0))

In [25]:
df = {'state': [], 'datetime': []}

for row in q:
    df['state'].append(row[1])
    df['datetime'].append(row[3])

df = pd.DataFrame(df)
df.head()

,state,datetime
0,Amazonas,2020-01-01 04:00:00
1,Amazonas,2020-01-01 04:00:00
2,Mato Grosso do Sul,2020-01-01 06:00:00
3,Pernambuco,2020-01-01 07:00:00
4,Rio Grande do Sul,2020-01-01 09:00:00
